In [2]:
import sys
from pathlib import Path

In [3]:
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

In [4]:
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import OllamaEmbeddings
from graph import EpilepsyState
from llms.groq_llm import get_groq_llm

In [5]:
from config import PINECONE_API_KEY

In [6]:
prompt_path = PROJECT_ROOT / "prompts" / "medical_rag_prompt.txt"

with open(prompt_path, "r", encoding="utf-8") as f:
    template_text = f.read()

In [ ]:
from langchain_core.prompts import PromptTemplate

In [8]:
prompt = PromptTemplate(
    template=template_text,
    input_variables=[
        "context",
        "epilepsy_presence",
        "seizure_phase",
        "seizure_type",
        "fusion_explanation",
    ],
)

In [ ]:
INDEX_NAME = "epilepsynexus"

In [10]:
pc = Pinecone(api_key=PINECONE_API_KEY)

In [1]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore

embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-en-v1.5",
    encode_kwargs={"normalize_embeddings": True}
)


d:\Epilepsy\EpilepsyNexus\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Chandana Aasritha\AppData\Local\Temp\ipykernel_2776\2326114763.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
Loading weights: 100%|██████████| 391/391 [00:01<00:00, 310.62it/s, Materializing param=pooler.dense.weight]                               
BertModel LOAD REPORT from: BAAI/bge-large-en-v1.5
Key                     | Status     |  | 
------------------------+-------

In [14]:
vectorstore = PineconeVectorStore(
    index_name=INDEX_NAME,
    embedding=embedding_model,
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})


In [22]:
llm = get_groq_llm()

In [20]:
def medical_rag_node(state: EpilepsyState) -> EpilepsyState:
    """
    Retrieves epilepsy knowledge and enriches model outputs.
    """

    # Build retrieval query
    query = f"""
    Epilepsy Presence: {state.epilepsy_presence}
    Seizure Phase: {state.seizure_phase}
    Seizure Type: {state.seizure_type}
    """

    # Retrieve documents from Pinecone
    docs = retriever.invoke(query)

    context = "\n\n".join([doc.page_content for doc in docs])

    # Run Groq LLM
    response = llm.invoke(
        prompt.format(
            context=context,
            epilepsy_presence=state.epilepsy_presence,
            seizure_phase=state.seizure_phase,
            seizure_type=state.seizure_type,
            fusion_explanation=state.fusion_explanation,
        )
    )

    # Store in state
    state.medical_context = response.content

    return state

In [23]:
if __name__ == "__main__":
    test_state = EpilepsyState(
        epilepsy_presence="yes",
        seizure_phase="ictal",
        seizure_type="focal",
        fusion_explanation="MRI abnormalities correlate with focal EEG findings.",
    )

    result = medical_rag_node(test_state)
    print(result.medical_context)


**Background on Epilepsy**  
- Epilepsy is a neurological condition in which a person experiences recurrent seizures.  
- About 60 % of people with epilepsy have focal seizures, which originate in one cerebral hemisphere.  

**Seizure Phase Characteristics**  
- **Ictal phase** – This is the period from the beginning of the seizure (including an aura, if present) to its end. It is the stage most people recognize as a seizure.  
- During the ictal phase, a person may display a variety of symptoms that differ between individuals. Typical manifestations can include repetitive small or large movements, unusual sensations, or emotional changes.  

**Seizure Type Characteristics**  
- **Focal seizures** start in a localized area of one side of the brain.  
- Awareness may be **preserved** (the person can remember the event and respond to the environment) or **impaired** (the person’s consciousness is affected).  
- An aura, when it occurs, is considered a focal aware seizure (formerly called